<a href="https://colab.research.google.com/github/RegenMedandAI/Machine-Learning-and-cancer-studies/blob/main/Exploratory_Data_Analysis_breast_cancer_GSE45827.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

i am doing a machine learning study on breast cancer data from Kaggle, https://www.kaggle.com/datasets/brunogrisci/breast-cancer-gene-expression-cumida, the csv was edited in r to include Gene_symbol (gene name alongside the microarray probe ID) and the tumour grade. This generated the file Breast_GSE45827p.csv, from which the study proceeds. The microarray transcription data for breast cancer has sample cataagories of "normal", of tumor grade 0; "Luminal_A", of tumor grade 1; "HER", of tumor grade 3; "basal", of tumor grade 3; and "Luminal B", of luminal grade 2. The samples are in columns starting at column3, column name is sample number, the first row are the catagories and the second row are the grades. The first column are the gene list as probeID and the second column are the gene list as Gene_Symbol. The EDA consists of NAn check in the expression data, PCA analysis of the sample groups, a heatmap of the control genes to see if normalisation is needed

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#load the data
data = pd.read_csv("Breast_GSE45827p.csv")

sample_ids = data.iloc[0, 2:153].index
categories = data.iloc[0, 2:153]
grades = data.iloc[1, 2:153]
sample_info = pd.DataFrame({
    'Sample_ID': sample_ids,
    'Category': categories,
    'Grade': grades
})
# Save the sample information to a CSV file
sample_info.to_csv('sample_info.csv', index=False)

# Extract gene information
gene_info = data.iloc[2:, :2]
gene_info.columns = ['ProbeID', 'GeneSymbol']
gene_info.reset_index(drop=True, inplace=True)

# Extract gene expression data
expression_data = data.iloc[2:, 2:153]
expression_data.columns = sample_ids
expression_data.set_index(gene_info['ProbeID'], inplace=True)

# Extract gene expression data
expression_data = data.iloc[2:, 2:153]
expression_data.columns = sample_ids

# Check for and handle any remaining NaN values
print("NaN values in expression data:", expression_data.isna().sum().sum())

In [ ]:
# Transpose the data so that samples are rows and genes are columns
expression_data_transposed = expression_data.T
# Perform PCA
scaler = StandardScaler()
scaled_expression = scaler.fit_transform(expression_data_transposed)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_expression)
# Print PCA results shape
print("PCA result shape:", pca_result.shape)


In [ ]:
# # Create DataFrame for PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'], index=sample_ids)
pca_df = pca_df.join(sample_info.set_index('Sample_ID'))

In [ ]:
# Print first few rows of PCA DataFrame
print("\nFirst few rows of PCA DataFrame:")
print(pca_df.head())

In [ ]:
# Print the range of PC1 and PC2
print("\nRange of PC1:", pca_df['PC1'].min(), "to", pca_df['PC1'].max())
print("Range of PC2:", pca_df['PC2'].min(), "to", pca_df['PC2'].max())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot histogram of PC1
sns.histplot(pca_df['PC1'], kde=True, ax=ax1)
ax1.set_title('Distribution of PC1')
ax1.set_xlabel('PC1 Values')

# Plot histogram of PC2
sns.histplot(pca_df['PC2'], kde=True, ax=ax2)
ax2.set_title('Distribution of PC2')
ax2.set_xlabel('PC2 Values')

plt.tight_layout()
plt.show()

In [ ]:
#variance explained analysis
import matplotlib.pyplot as plt

# Perform PCA with more components
pca_full = PCA()
pca_full.fit(scaled_expression)

# Calculate cumulative variance explained
cumulative_variance_ratio = np.cumsum(pca_full.explained_variance_ratio_)

# Plot variance explained
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, 'bo-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance Ratio by Principal Components')
plt.axhline(y=0.8, color='r', linestyle='--', label='80% Variance Explained')
plt.legend()
plt.show()

# Print number of components needed to explain 80% of variance
n_components_80 = np.where(cumulative_variance_ratio >= 0.8)[0][0] + 1
print(f"Number of components needed to explain 80% of variance: {n_components_80}")

In [ ]:
# prompt: open Breast_GSE45827p.csv as Breast_GSE45827p dataframe, number the rows from 1 upwards make column names the first row (row1)  for  Breast_GSE45827p then print, save Breast_GSE45827q
#tidying dataframe errors
import pandas as pd

# Open Breast_GSE45827p.csv as a DataFrame
Breast_GSE45827p = pd.read_csv("Breast_GSE45827p.csv", header=None)

# Set the first row as column names
Breast_GSE45827p.columns = Breast_GSE45827p.iloc[0]

# Remove the first row (which is now the header)
Breast_GSE45827p = Breast_GSE45827p.iloc[1:]

# Reset the index to start from 1
Breast_GSE45827p.index = np.arange(1, len(Breast_GSE45827p) + 1)

# Print the DataFrame
print(Breast_GSE45827p)

# Save the DataFrame to a new CSV file
Breast_GSE45827p.to_csv('Breast_GSE45827q.csv')


explain PCA analysis for gene expression
ChatGPT said:
ChatGPT
Principal Component Analysis (PCA) is a dimensionality reduction technique used in gene expression analysis to identify patterns and simplify complex datasets. In gene expression studies, where thousands of genes are analyzed across multiple samples, PCA helps reduce the data into a smaller set of uncorrelated variables called principal components (PCs). These PCs capture the most variance in the data, with the first few components accounting for the majority of variation in gene expression. By plotting samples against these PCs, researchers can identify clustering patterns, such as how different experimental conditions, cell types, or disease states separate. PCA helps reveal underlying structures in the data, making it easier to identify relationships between genes and conditions without getting lost in the high dimensionality of the data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
Breast_GSE45827p = pd.read_csv('Breast_GSE45827p.csv')

# 1. Skip row numbering (data already has rownames)
# 2. Extract relevant parts of the data
sample_ids6 = Breast_GSE45827p.iloc[0, 2:].values  # Sample IDs in row 0 (excluding name)
sample_grades6 = Breast_GSE45827p.iloc[1, 2:].values  # Sample types in row 1 (excluding name)
probe_ids6 = Breast_GSE45827p.iloc[3:, 0].values         # Probe IDs starting from row 4 (index)
gene_symbols6 = Breast_GSE45827p.iloc[3:, 1].values      # Gene symbols starting from row 4
gene_expression6 = Breast_GSE45827p.iloc[3:, 2:].T    # Transpose to have samples as rows

# 3. Print the first few sample types and grades
print("First few sample types:")
print(sample_grades6[:5])


# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Grade': sample_grades6,

})

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Grade', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast Cancer Gene Expression Data (Breast_GSE45827p) by Grade')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Cancer Grade')
plt.grid(True)
plt.show()


# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
Breast_GSE45827p = pd.read_csv('Breast_GSE45827p.csv')

# 1. Skip row numbering (data already has rownames)
# 2. Extract relevant parts of the data
sample_ids6 = Breast_GSE45827p.iloc[0, 2:].values  # Sample IDs in row 0 (excluding name)
sample_types6 = Breast_GSE45827p.iloc[1, 2:].values  # Sample types in row 1 (excluding name)
probe_ids6 = Breast_GSE45827p.iloc[3:, 0].values         # Probe IDs starting from row 3 (index)
gene_symbols6 = Breast_GSE45827p.iloc[3:, 1].values      # Gene symbols starting from row 3
gene_expression6 = Breast_GSE45827p.iloc[3:, 2:].T    # Transpose to have samples as rows

# 3. Print the first few sample types
print("First few sample types (Cancer types):")
print(sample_ids6[:5])


# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Type': sample_ids6,
    })

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Type
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Type', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast Cancer Gene Expression Data (Breast_GSE45827p) by Type')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Cancer Type')
plt.grid(True)
plt.show()


# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

In [ ]:
# Hierarchical Clustering with Heatmap of AFFX and controls, test whether normalisation is required

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('Breast_GSE45827p.csv')

# Extract sample data (columns 3 to 153, rows 3 to 54677) and convert to float
sample_data = df.iloc[2:54677, 2:153].astype(float)

# Extract probe IDs and gene symbols
probe_ids = df.iloc[2:54677, 0]
gene_symbols = df.iloc[2:54677, 1]

# Extract sample names, types, and grades
sample_names = df.columns[2:153]
sample_types = df.iloc[0, 2:153]
sample_grades = df.iloc[1, 2:153]

# Filter for AFFX probes
affx_mask = probe_ids.str.contains('AFFX', case=False, na=False)
affx_data = sample_data[affx_mask]
affx_probe_ids = probe_ids[affx_mask]

# Create a DataFrame with AFFX probe data
affx_df = pd.DataFrame(affx_data.values, index=affx_probe_ids, columns=sample_names)

# Create the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(affx_df, cmap='coolwarm', center=0, cbar_kws={'label': 'Expression Value'})
plt.title('Heatmap of AFFX Probes Across Samples')
plt.xlabel('Samples')
plt.ylabel('AFFX Probe IDs')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Total number of AFFX probes: {len(affx_probe_ids)}")
print("\nFirst 10 AFFX probe IDs:")
print(affx_probe_ids[:10].tolist())

# Calculate and print average expression for each AFFX probe
avg_expression = affx_df.mean(axis=1).sort_values(ascending=False)
print("\nTop 10 AFFX probes by average expression:")
print(avg_expression.head(10))

print("\nBottom 10 AFFX probes by average expression:")
print(avg_expression.tail(10))

In [ ]:
#differential analysis- test gene identification and analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection

# Load the data
data = pd.read_csv('Breast_GSE45827p.csv', header=None)

# Extract sample information
sample_info = pd.DataFrame({
    'Sample': data.iloc[0, 2:152],
    'Type': data.iloc[1, 2:152],
    'Grade': data.iloc[2, 2:152]
})

# Extract gene expression data and convert to float
expression_data = data.iloc[2:54676, 2:152].astype(float).T
expression_data.columns = data.iloc[2:54676, 0]  # Set probe_id as column names
expression_data.index = sample_info.index

# Create a dictionary to map probe_id to gene symbol
gene_symbol_dict = dict(zip(data.iloc[2:54676, 0], data.iloc[2:54676, 1]))

def perform_de_analysis(normal_samples, comparison_samples, comparison_name):
    results = []
    for probe in expression_data.columns:
        t_stat, p_val = stats.ttest_ind(normal_samples[probe], comparison_samples[probe])
        results.append((probe, gene_symbol_dict[probe], t_stat, p_val))

    de_results = pd.DataFrame(results, columns=['Probe_ID', 'Gene_Symbol', 't_statistic', 'p_value'])
    de_results['adjusted_p_value'] = fdrcorrection(de_results['p_value'])[1]
    de_results = de_results.sort_values('adjusted_p_value')

    print(f"\nTop 10 differentially expressed genes between Normal and {comparison_name}:")
    print(de_results.head(10))

    # Volcano plot
    plt.figure(figsize=(10, 6))
    plt.scatter(de_results['t_statistic'], -np.log10(de_results['adjusted_p_value']), alpha=0.5)
    plt.xlabel('t-statistic')
    plt.ylabel('-log10(adjusted p-value)')
    plt.title(f'Volcano Plot: Normal vs {comparison_name}')
    plt.savefig(f'volcano_plot_normal_vs_{comparison_name}.png')
    plt.close()

    return de_results

# Perform differential expression analysis for each type against normal
normal_samples = expression_data[sample_info['Type'] == 'normal']

for comparison_type in ['basal', 'HER', 'luminal_A', 'luminal_B']:
    comparison_samples = expression_data[sample_info['Type'] == comparison_type]
    results = perform_de_analysis(normal_samples, comparison_samples, comparison_type)
    results.to_csv(f'de_results_normal_vs_{comparison_type}.csv', index=False)

print("Analysis complete. Results have been saved to CSV files and volcano plots have been generated.")

# Add some data validation checks
print("\nData Validation:")
print(f"Shape of expression_data: {expression_data.shape}")
print(f"Data types in expression_data: {expression_data.dtypes.value_counts()}")
print(f"Any NaN values in expression_data: {expression_data.isna().any().any()}")
if expression_data.isna().any().any():
    print(f"Number of NaN values: {expression_data.isna().sum().sum()}")

In [ ]:
#(optional) produce csv of differentially expressed genes for later analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection

# Load the data
data = pd.read_csv('Breast_GSE45827p.csv', header=None)

# Extract sample information
sample_info = pd.DataFrame({
    'Sample': data.iloc[0, 2:152],
    'Type': data.iloc[1, 2:152],
    'Grade': data.iloc[2, 2:152]
})

# Extract gene expression data and convert to float
expression_data = data.iloc[2:54676, 2:152].astype(float).T
expression_data.columns = data.iloc[2:54676, 0]  # Set probe_id as column names
expression_data.index = sample_info.index

# Create a dictionary to map probe_id to gene symbol
gene_symbol_dict = dict(zip(data.iloc[2:54676, 0], data.iloc[2:54676, 1]))

def perform_de_analysis(normal_samples, comparison_samples, comparison_name):
    results = []
    for probe in expression_data.columns:
        normal_mean = normal_samples[probe].mean()
        comparison_mean = comparison_samples[probe].mean()
        fold_change = comparison_mean - normal_mean  # Log2 fold change since data is likely log-transformed
        t_stat, p_val = stats.ttest_ind(normal_samples[probe], comparison_samples[probe])
        results.append((probe, gene_symbol_dict[probe], t_stat, p_val, fold_change))

    de_results = pd.DataFrame(results, columns=['Probe_ID', 'Gene_Symbol', 't_statistic', 'p_value', 'fold_change'])
    de_results['adjusted_p_value'] = fdrcorrection(de_results['p_value'])[1]

    # Filter for upregulated genes (positive fold change)
    upregulated_genes = de_results[de_results['fold_change'] > 0].sort_values('adjusted_p_value')

    # Add comparison type to the results
    top_50_results = upregulated_genes.head(50).copy()
    top_50_results['Comparison'] = comparison_name

    # Volcano plot
    plt.figure(figsize=(10, 6))
    plt.scatter(de_results['fold_change'], -np.log10(de_results['adjusted_p_value']), alpha=0.5)
    plt.scatter(top_50_results['fold_change'],
                -np.log10(top_50_results['adjusted_p_value']),
                color='red', alpha=0.5)
    plt.xlabel('Log2 Fold Change')
    plt.ylabel('-log10(adjusted p-value)')
    plt.title(f'Volcano Plot: Normal vs {comparison_name}\nTop 50 upregulated genes highlighted in red')
    plt.savefig(f'volcano_plot_normal_vs_{comparison_name}.png')
    plt.close()

    return top_50_results

# Perform differential expression analysis for each type against normal
normal_samples = expression_data[sample_info['Type'] == 'normal']
all_results = []

for comparison_type in ['basal', 'HER', 'luminal_A', 'luminal_B']:
    comparison_samples = expression_data[sample_info['Type'] == comparison_type]
    results = perform_de_analysis(normal_samples, comparison_samples, comparison_type)
    all_results.append(results)

# Combine all results into a single DataFrame
combined_results = pd.concat(all_results, ignore_index=True)

# Reorder columns for better readability
column_order = ['Comparison', 'Gene_Symbol', 'Probe_ID', 'fold_change', 't_statistic', 'p_value', 'adjusted_p_value']
combined_results = combined_results[column_order]

# Save combined results to CSV
combined_results.to_csv('all_top50_upregulated_genes.csv', index=False)

print("\nAnalysis complete. All top 50 differentially expressed genes for each comparison have been saved to 'all_top50_upregulated_genes.csv'")

# Data validation and summary
print("\nSummary of results:")
print(combined_results['Comparison'].value_counts())
print("\nNumber of unique genes:", combined_results['Gene_Symbol'].nunique())

# Data validation checks
print("\nData Validation:")
print(f"Shape of expression_data: {expression_data.shape}")
print(f"Shape of combined results: {combined_results.shape}")
print(f"Any NaN values in results: {combined_results.isna().any().any()}")

In [ ]:
# Save combined results to CSV
combined_results.to_csv('all_top50_upregulated_genes.csv', index=False)

A volcano plot is a scatter plot used in differential gene expression analysis to visualize the relationship between fold-change and statistical significance of genes. On the x-axis, it shows the log2 fold-change, representing how much a gene’s expression level changes between two conditions, while the y-axis shows the -log10 of the p-value, indicating the statistical significance of this change. Genes with large fold-changes and low p-values appear at the top edges of the plot, representing the most differentially expressed genes. It helps to quickly identify genes that are both biologically significant (large fold-change) and statistically significant (low p-value).

In [ ]:
# prompt: view volcano_plot_normal_vs_basal

from IPython.display import Image
Image(filename='volcano_plot_normal_vs_HER.png')
